# Preprocessing CBN Data

In [ ]:
from cng.utils import *
from utils import *
from h3_utils import *

import os
os.chdir('../data/')

s3 = s3_client()
duckdb_install_h3()

## Counties **

In [ ]:
%%time 
con = ibis.duckdb.connect('counties',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Counties'
name = 'CA_counties'

# unzip(s3, folder = folder, file = '30x30_Counties.zip')
cols = process_vector(s3, folder = folder, file = f"{name}.shp")
# convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")

convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 8)

## Climate Zones **

In [ ]:
%%time 
con = ibis.duckdb.connect('climate_zones',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Climate_zones'
name = 'climate_zones_10'
# download(s3, folder = folder, file = 'clusters_10.tif')
cols = process_raster(s3, folder = folder, file = 'clusters_10.tif', file_name = f"{name}.tif")
convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols = cols,
          zoom = 8)


## Ecoregions **

In [ ]:
%%time 
con = ibis.duckdb.connect('ecoregion',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Ecoregion'
name = 'ACE_ecoregions'

# unzip(s3, folder = folder, file = '30x30_Ecoregions.zip')
cols = process_vector(s3, folder = folder, file = f"{name}.shp")

convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 8)

## Habitat

#### 13 class major habitat types **

In [ ]:
%%time
con = ibis.duckdb.connect('habitat',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Habitat'
name = 'fveg22_1'
# unzip(s3, folder = folder, file = 'fveg221gdb.zip')

# command = [
#     "gdalwarp",
#     "-of", "GTiff",
#     'fveg22_1.gdb',
#     'fveg22_1.tif' 
#     ]

# subprocess.run(command, check=True)
cols = process_raster(s3, folder = folder, file = f"{name}.tif")
# upload(folder = folder, file = f'{name}_processed.tif.aux.xml')

convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols = cols,
          zoom = 8)

#### 60+ class habitat types

## ACE Biodiversity

In [ ]:
%%time 
con = ibis.duckdb.connect('ace',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'ACE_biodiversity'
name = 'ACE_terrestrial_biodiversity_summary_ds2739'

# download(s3, folder = folder, file = 'Terrestrial_Biodiversity_Summary_-_ACE_[ds2739].geojson',
         # file_name = f"{name}.geojson")

cols = process_vector(s3, folder = folder, file = f"{name}.geojson")
cols = [item for item in cols if item not in ["Hex_ID","Shape__Area","Shape__Length"]]
# convert_pmtiles(con, s3, folder = folder, file = f"{name}.geojson")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 8)
# gdf = gpd.read_parquet(f"{name}.parquet")


#### ACE BioRank and Rare Rank 

In [ ]:
# Filter data to rank 5.
ACE_rank_files = ['ACE_biorank_statewide','ACE_biorank_ecoregion',
                  'ACE_rarerank_statewide','ACE_rarerank_ecoregion']
             
ACE_rank_cols =  ['BioRankSW','BioRankEco','RarRankSW','RarRankEco']   

for col,name in zip(ACE_rank_cols,ACE_rank_files):
    cols = ['OBJECTID', 'Hex_ID', 'Eco_Sect', 'Eco_Name',
            'County', 'Shape__Area', 'Shape__Length', 'geometry']
    cols.append(col) #select only the cols we want + the new col. 
    rank_df = gdf[gdf[col]==5][cols]# filter ranks = 5
    cols = process_vector(s3, folder = 'ACE_biodiversity/'+name, file = name+'.parquet',gdf = rank_df)
    convert_pmtiles(con, s3, folder ='ACE_biodiversity/'+name, file = name+'.parquet')


#### Other ACE Biodiversity **

In [ ]:
ACE_files = ['ACE_amphibian_richness','ACE_reptile_richness',
            'ACE_bird_richness','ACE_mammal_richness',
            'ACE_rare_amphibian_richness','ACE_rare_reptile_richness',
            'ACE_rare_bird_richness','ACE_rare_mammal_richness',
            'ACE_endemic_amphibian_richness','ACE_endemic_reptile_richness',
            'ACE_endemic_bird_richness','ACE_endemic_mammal_richness']

ACE_cols = ['NtvAmph','NtvRept','NtvBird','NtvMamm','RarAmph','RarRept','RarBird','RarMamm',
            'AmphEndem','ReptEndem','BirdEndem','MammEndem']

for col,name in zip(ACE_cols,ACE_files):
    cols = ['OBJECTID', 'Hex_ID', 'Eco_Sect', 'Eco_Name',
            'County', 'Shape__Area', 'Shape__Length', 'geometry']
    cols.append(col) #select only the cols we want + the new col. 
    if name in ['NtvAmph','NtvRept','NtvBird','NtvMamm']:
        percentile = 0.8
    else: 
        percentile = 0.95
    threshold = gdf[col].quantile(percentile)
    ace = gdf[gdf[col]>=threshold][cols]
    cols = process_vector(s3, folder = 'ACE_biodiversity/'+name, file = name+'.parquet',gdf = ace)
    convert_pmtiles(con, s3, folder ='ACE_biodiversity/'+name, file = name+'.parquet')


# calculate 80% percentile, filter to those >= threshold. 
# subset to calculate acres within each network, % of feature conserved and % of network 

## Biodiversity

#### Plant richness **

In [ ]:
%%time 
con = ibis.duckdb.connect('plant',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Biodiversity_unique/Plant_richness'
name = 'species_D'

# download(s3, folder = folder, file = f"{name}.tif")
cols = filter_raster(s3, folder = folder, file = f"{name}.tif", percentile = 80)
convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols = cols, zoom = 8)

#### Rarity-weighted endemic plant richness **

In [ ]:
%%time 
con = ibis.duckdb.connect('end_plant',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Biodiversity_unique/Rarityweighted_endemic_plant_richness'
name = 'endemicspecies_E'

download(s3, folder = folder, file = f"{name}.tif")
cols = filter_raster(s3, folder = folder, file = f"{name}.tif", percentile = 80)
convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols = cols, zoom = 8)

#### Abundance for 26 bird species

## Connectivity and Resilience
#### Resilient Connected Network - all categories **

In [ ]:
%%time 
con = ibis.duckdb.connect('CRN',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Connectivity_resilience/Resilient_connected_network_allcategories'
name = 'rcn_wIntactBioCat_caOnly_2020-10-27'

cols = process_raster(s3, folder = folder, file = f"{name}.tif")
convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols = cols, 
           zoom = 8)

#### Present day connectivity - all categories

#### Climate migration routes

## Freshwater Resources

#### Freshwater species richness

#### Wetlands **

In [ ]:
%%time 
con = ibis.duckdb.connect('wetlands',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Freshwater_resources/Wetlands'
name = 'CA_wetlands'

# only pick a subset 
unzip(s3, folder = folder, file = 'CA_geodatabase_wetlands.zip')
gdf = gpd.read_file('CA_geodatabase_wetlands.gdb')
wetlands = ['Freshwater Emergent Wetland', 'Freshwater Forested/Shrub Wetland', 'Estuarine and Marine Wetland']
gdf = gdf[gdf['WETLAND_TYPE'].isin(wetlands)]

cols = process_vector(s3, folder = folder, file = f"{name}.parquet", gdf = gdf)
cols = [item for item in cols if item not in ['ACRES','Shape_Length','Shape_Area','__index_level_0__']]
convert_pmtiles(con, s3, folder =folder, file = f"{name}.parquet")
geom_to_h3(con, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 8)


#### Groundwater dependent ecosystems

#### Streams by order

#### Perennial streams

#### Fish passage barriers

## NBS and Agriculture

#### Drinking water source watersheds

#### Farmland + Land suitable for grazing **

In [ ]:
%%time 
con = ibis.duckdb.connect('farm',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'NBS_agriculture/Farmland_all'
name = 'Important_Farmland_2018'
# unzip(s3, folder = folder, file = f"{name}.zip")
cols = process_vector(s3, folder = folder, file = f"{name}.gdb",crs = "epsg:4326")
cols = [item for item in cols if item not in ['Shape_Length','Shape_Area']]

# convert_pmtiles(con, s3, folder = folder, file =f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 8)

# only pick a subset 
folder = 'NBS_agriculture/Farmland_all/Farmland'
name = 'Farmland_2018'
# gdf = gpd.read_file('Important_Farmland_2018.gdb')
# farmland_type = ['P','S','L','U'] # prime, statewide importance, local importance, unique
# gdf_farmland = gdf[gdf['polygon_ty'].isin(farmland_type)]
# cols = process_vector(s3, folder = folder, file = f"{name}.parquet", gdf = gdf_farmland)
# convert_pmtiles(con, s3, folder = folder, file =f"{name}.parquet")

# grazing lands 
folder = 'NBS_agriculture/Farmland_all/Lands_suitable_grazing'
name = 'Grazing_land_2018'
# gdf_grazing = gdf[gdf['polygon_ty'] == 'G']
# cols = process_vector(s3, folder = folder, file = f"{name}.parquet", gdf = gdf_grazing)
# convert_pmtiles(con, s3, folder = folder, file =f"{name}.parquet")


#### Carbon storage **

## Climate Risks

#### Fire perimeters **

Only YEAR >= 2014. 

In [ ]:
%%time 
con = ibis.duckdb.connect('fire',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Climate_risks/Historical_fire_perimeters'
name = 'calfire_2023'

unzip(s3, folder = folder, file = 'fire23-1gdb.zip')
gdf = gpd.read_file('fire23_1.gdb')
# gdf = gdf[~gdf['YEAR_'].isna()]
# gdf['YEAR_'] = gdf['YEAR_'].astype('int64')
# gdf = gdf[gdf['YEAR_']>=2014]
cols = process_vector(s3, folder = folder, file = f"{name}.parquet", gdf = gdf)
cols = [item for item in cols if item not in ['Shape_Length','Shape_Area']]

# convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")

convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 8)

#### Flood hazard zones **

#### Sea level rise

#### Mid-century habitat climate exposure **

In [ ]:
'''
First mask out non-natural lands.
A binary natural vs. non-natural  land mask is included in the data package. 
Use the combined group of all values < 0 and >=0.95 as exposed.  
Do seperately for both climate models - CNRM and MIROC.
'''

unzip(s3, folder = 'Climate_risks/Mid-century_habitat_climate_exposure', file = 'Midcentury_habitat_climate_exposure.zip')

# still need to do 

## Progress data - newly protected

#### Newly counted

In [ ]:
%%time 
con = ibis.duckdb.connect('new_land',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Progress_data_new_protection/Newly_counted_lands'
name = 'newly_counted_lands_2024'


# unzip(s3, folder = folder, file = f"{name}.shp.zip")
cols = process_vector(s3, folder = folder, file = f"{name}.shp",crs = "epsg:4326")
cols = [item for item in cols if item not in ['Shape_Leng', 'Shape_Area']]

# convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 8)


#### DAC **

In [ ]:
%%time 
con = ibis.duckdb.connect('dac',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Progress_data_new_protection/DAC'
name = 'DAC_2022'

unzip(s3, folder = folder, file = 'sb535dacgdbf2022gdb.zip')
cols = process_vector(s3, folder = folder, file = 'SB535DACgdb_F_2022.gdb', file_name = f"{name}.parquet")
convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")


#### Priority populations

In [ ]:
%%time 
con = ibis.duckdb.connect('priority_pop',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Progress_data_new_protection/Priority_populations'
name = 'CalEnviroScreen4'
# unzip(s3, folder = folder, file = 'Priority Populations 4.0 Geodatabase.zip')

gdf = (con.read_geo('Priority Populations 4.0 Combined Layer.gdb')
       .mutate(id=ibis.row_number().over()) #making a unique id 
      ).execute().set_crs('EPSG:3857')

cols = process_vector(s3, folder = folder, file = 'Priority Populations 4.0 Combined Layer.gdb',
               file_name = f"{name}.parquet", gdf = gdf)
cols = [item for item in cols if item not in ['Shape_Length','Shape_Area']]

# convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 8)


#### Low income communities **

In [ ]:
con = ibis.duckdb.connect('low',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Progress_data_new_protection/Low_income_communities'
name = 'low_income_CalEnviroScreen4'

unzip(s3, folder = folder, file = 'Priority Populations 4.0 Geodatabase.zip')

gdf = gpd.read_file('Priority Populations 4.0 Combined Layer.gdb')
gdf = gdf[gdf['Designatio'] =='Low-income community']
cols = process_vector(s3, folder = folder, file = f"{name}.parquet", gdf = gdf)
convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")

## Base layer for denominator 

In [ ]:
con = ibis.duckdb.connect('base_layer',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Progress_data_new_protection/Land_Status_Zone_Ecoregion_Counties'
name = 'all_regions_reGAP_county_eco'

# unzip(s3, folder = folder, file = 'Land_Status_Zone_Ecoregion_Counties.shp.zip')
cols = process_vector(s3, folder = folder, file = 'Land_Status_Zone_Ecoregion_Counties.shp',
               file_name = f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, zoom = 5)

# convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")

# CA Nature data

In [ ]:
%%time 
con = ibis.duckdb.connect('ca_30x30_base',extensions = ["spatial", "h3"])
set_secrets(con)

# file = 'ca-30x30-base.parquet'
folder = "CA_Nature/2024/Preprocessing"
name = 'ca-30x30-base'
# download(s3, folder = folder, file = f"{name}.parquet")

# cols = process_vector(s3, folder = folder, file = f"{name}.parquet", crs="EPSG:4326")
# convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols = cols, 
           # zoom = 8)
convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")


# CPAD

In [ ]:
con = ibis.duckdb.connect('cpad',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'CPAD'
name = 'cced_2024b_release'

# unzip(s3, folder = folder, file = f"{name}.shp.zip")
# cols = process_vector(s3, folder = folder, file = f"{name}.shp", crs="EPSG:3310")
# convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")
cols = process_vector(s3, folder = folder, file = f"{name}.shp", crs="EPSG:4326")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= cols, zoom = 8)

name = 'cpad_2024b_release'
# unzip(s3, folder = folder, file = f"{name}.shp.zip")
# cols = process_vector(s3, folder = folder, file = f"{name}.shp", crs="EPSG:3310")
# convert_pmtiles(con, s3, folder = folder, file = f"{name}.parquet")
cols = process_vector(s3, folder = folder, file = f"{name}.shp", crs="EPSG:4326")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= cols, zoom = 8)